# 유튜브 크롤링 후 DB 적재 자동화

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import psycopg2
import re

In [4]:
#스크롤 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

# 해시태그 함수
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        #print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        #print(hash_tag)
        hashtag_list.append(hash_tag)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

뷔스티에 원피스, 롱패딩, 숏패딩, 롱코트, 숏코트, 잠옷, 홈웨어, 트위드자켓, 블레이저 자켓, 싱글 자켓, 크롭 자켓, 레더 자켓, 루즈핏, 테일러드 자켓, 린넨자켓, 반팔자켓, 라이더자켓, 오버핏자켓, 청자켓, 니트가디건, 볼레로, 아가일, 크롭가디건, 반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', 후드집업, 투웨이집업, 여름아우터, 간절기아우터, 겨울아우터, 기모맨투맨, 니트맨투맨, 크롭 맨투맨', '남자 맨투맨', '반집업', 후드티, 남자후드, 꾸안꾸, 플리스, 반팔티, 크롭티, 블라우스, 와이셔츠, 셔츠, 체크셔츠, 타이다이, 옥스퍼드셔츠, 남방, 슬리브티, 니트, 남자니트, 조끼, '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', '미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', '미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', '린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울', '겨울룩북', '겨울옷쇼핑', '겨울코디', '겨울하울'

keyword_list = ['블라인드', '롤스크린', '커튼', '러그', '카페트', '매트', '쿠션', '방석', '소파커버', '조명', 'led평판등', '천장등', '장스탠드', '단스탠드', '데스크스탠드', '무드등', '장식조명', '벽조명', '센서등', ]

In [1]:
keyword_list = ['형광등', '집꾸미기', '방꾸미기', '인테리어']

In [7]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

for keyword in keyword_list :

    # 키워드를 검색한 유튜브 주소 접속
    url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
    driver.get(url)

    #사이트 최대화
    driver.maximize_window()

    # 필터 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
    time.sleep(1)

    # 구분 -> 동영상 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

    # 업로드 날짜 -> 올해 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()


    # 스크롤 다운 50번 실행
    body = driver.find_element_by_tag_name('body')
    body.send_keys(Keys.PAGE_DOWN)

    for i in range(1,50):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    name = soup.select('a#video-title')
    video_url = soup.select('a#video-title')

    video_num_list = [] #영상번호
    name_list = [] #영상제목
    url_list = [] #영상주소

    for i in range(len(name)):
        video_num_list.append(i+1)
        name_list.append(name[i].text.strip())

    for i in video_url:
        url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

    print(keyword, '영상 개수: ', len(name_list))

    # 업로드일 & 해시태그 가져오기
    day_list = [] #업로드일
    hashtag_list = [] #해시태그

    for i in range(0,len(name_list)):

        print(i+1, end=' ')
        
        if url_list[i].split('/')[3] == 'shorts':  #쇼츠

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            
            driver.get(url_list[i])

            time.sleep(2)

            #더보기 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
            time.sleep(2)

            #설명 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
            time.sleep(2)

            #업로드일 정보 가져오기
            day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()
            day_list.append(day)

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
            get_hashtag(outline_test)

        else :

            driver.get(url_list[i])
                
            time.sleep(2)    

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')

            time.sleep(2)  

            #업로드일 정보 가져오기
            #day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
            day = driver.find_element_by_id("info-strings").text.split('\n')[0]
            day_list.append(day)

            if day == "" :
                print('error in getting day')
                break

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
            get_hashtag(outline_test)

    print('끝')

    day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
    for day in day_list : 
        day = day.replace('.', '')
        day = day.replace('최초 공개: ', '')
        day = day.replace('실시간 스트리밍 시작일: ', '')
        day = day.replace(' ', '-')
        day_list2.append(day)

    #df화
    youtubeDic = {
        '제목': name_list,
        '업로드일': day_list2,
        '주소': url_list,
        '해시태그' : hashtag_list
    }

    youtubeDf = pd.DataFrame(youtubeDic)

    # datetime 타입으로 변환
    youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

    # 중복값제거
    youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

    print(youtubeDf.tail())


    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    table = 'yt_interior'

    #table에 데이터 넣기
    for i in range(len(youtubeDf)) :
        cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
        conn.commit()

    #db 연결 종료
    conn.cursor().close()
    conn.close()
    

WebDriverException: Message: chrome not reachable
  (Session info: chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x005FD953+2414931]
	Ordinal0 [0x0058F5E1+1963489]
	Ordinal0 [0x0047C580+836992]
	Ordinal0 [0x004701E2+786914]
	Ordinal0 [0x00470A18+789016]
	Ordinal0 [0x004722F2+795378]
	Ordinal0 [0x0046BCA9+769193]
	Ordinal0 [0x0047DA30+842288]
	Ordinal0 [0x004D3FF2+1196018]
	Ordinal0 [0x004C42B6+1131190]
	Ordinal0 [0x0049E860+976992]
	Ordinal0 [0x0049F756+980822]
	GetHandleVerifier [0x0086CC62+2510274]
	GetHandleVerifier [0x0085F760+2455744]
	GetHandleVerifier [0x0068EABA+551962]
	GetHandleVerifier [0x0068D916+547446]
	Ordinal0 [0x00595F3B+1990459]
	Ordinal0 [0x0059A898+2009240]
	Ordinal0 [0x0059A985+2009477]
	Ordinal0 [0x005A3AD1+2046673]
	BaseThreadInitThunk [0x77B5FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77CA7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77CA7A4E+238]


In [53]:
day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('최초 공개일: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

    #df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

    # datetime 타입으로 변환
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

    # 중복값제거
youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

print(youtubeDf.tail())


    #postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_interior'

    #table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

    #db 연결 종료
conn.cursor().close()
conn.close()

                                                    제목       업로드일  \
132                                        멀티미니발판 땅꽂이용 2022-01-31   
133                광고# 1분안에 스탠드조명 가성비 상품 추천순위 top10 정리 2022-04-29   
134            버섯 독서등 책상 스탠드 엔틱 침실 침대 독서실 인테리어 단스탠드 조명 2021-09-25   
135  2022년형 김치냉장고 비교 1탄 "스탠드형, 뚜껑형 우리집에 맞는 김치냉장고?" ... 2021-09-27   
136  실내 정원을 위한 식물등을 추천합니다. 제가 제돈 주고 두 번이나 구매한 식물등은 ... 2021-10-11   

                                              주소  \
132  https://www.youtube.com/watch?v=rkTjFDg8y-4   
133  https://www.youtube.com/watch?v=Wx3HMceI-8c   
134  https://www.youtube.com/watch?v=G6vsYcTuPT8   
135  https://www.youtube.com/watch?v=u-_XQ272H5g   
136  https://www.youtube.com/watch?v=ES_0Z5A9Ue4   

                                                  해시태그  
132                                                     
133  #스피아노 #마켓비 #더숨 #코멧 #씨티오 #스피아노 #스피아노 #마켓비 #스피아노...  
134                                                     
135                              #김치냉장고비교 #전

In [14]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(hashtag_list)) :
    a = len(hashtag_list[i])

    if a > b :
        b = a
        print(i, b)

print(hashtag_list[84][:202])

4 88
29 294
84 521
#옷만들기 #명품 #원피스 #홀터넥비키니수영복 #수영복코디 #수영복만들기 #여름코디 #올인원수영복 #명품 #수영 #홀터넥비키니수영복 #수영복코디 #수영복만들기 #여름코디 #데님반바지 #수영복 #청바지 #데님청수영복 #호피무늬 #비키니수영복 #랩스커트 #숄 #인싸템 #원피스리폼 #호피무늬수영복 #여름코디 #비키니룩북 #수영복리폼 #여름 #호피무늬코디 #수영복리폼 


In [478]:
hashtag_list[112] = hashtag_list[112][:201]

# #############여기까지만#############

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [65]:
#db 연결 종료
conn.cursor().close()
conn.close()

create table yt_clothes (
	id serial,
	title varchar(100),
	date date,
	url varchar(200));